# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |  Regina Tamayo León |
| **Fecha**      | 16 feb 2026  |
| **Expediente** |757857   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

es para validar si el modelo realmente aprendió o si solo memorizó. queremos saber si estamos generalizando o no.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Entrenar con todo sin haber probado antes con un test sería un error porque no tienes forma de saber si van a ocurrir errores. puede que el modelo se haya prendido la lógica general pero al momento de hacer pruebas fallará.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

**EDA**

In [6]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

df=pd.read_excel("Motor Trend Car Road Tests.xlsx")
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


*Eliminar columna*

In [7]:
df.drop(columns=["model"], inplace=True)


*variables*

In [8]:
X=df.drop(columns=["mpg"]).values
y=df["mpg"].values
model=LinearRegression()
mses=[]

*Leave-One-Out Cross Validation*

In [9]:
for i in range(len(df)):
    X_train=np.delete(X,i,axis=0)
    y_train=np.delete(y,i)
    X_test=X[i].reshape(1,-1)
    y_test=y[i].reshape(1,)
    
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    
    mse=mean_squared_error(y_test,pred)
    mses.append(mse)

*Resultados*

In [10]:

print("MSE promedio:", np.mean(mses))
print("Desviación estándar:", np.std(mses))

MSE promedio: 12.181558006901959
Desviación estándar: 17.06739987188857


Interpreta.

Un valor de 12.18 indica que, en promedio, el error al cuadrado no es extremadamente alto. Esto significa que el modelo se equivoca aproximadamente ±3.5 mpg en sus predicciones.

La desviación estándar (17.07) es incluso mayor que el promedio. Eso significa: que algunos autos generan errores muy pequeños y otros generan errores muy grandes
el modelo no es estable.

no generaliza de forma constante

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [11]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [12]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

model = LinearRegression()

kf = KFold(n_splits=10, shuffle=True, random_state=42)

mses = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mses.append(mse)

print("MSE por fold:", mses)
print("MSE promedio:", np.mean(mses))
print("Desviación estándar:", np.std(mses))


MSE por fold: [0.5590019245025518, 0.5531232970331553, 0.4811612796368764, 0.5742036665176022, 0.488975837819139, 0.5283881757036992, 0.5511782380280766, 0.4748396129801818, 0.5643692207196765, 0.5500253663196355]
MSE promedio: 0.5325266619260595
Desviación estándar: 0.03522806775487864


Interpreta.

El MSE = 0.5325 indica el error cuadrático medio en la predicción del valor de casas.
la raíz del error cuádratico medio es 0.73,entonces el error promedio real es $73,000
en promedio, el modelo se equivoca por alrededor de 73 mil dólares al predecir el precio medio de una vivienda.

La desviación estándar es muy baja
El modelo produce resultados consistentes y generaliza bien

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3